**Doğrusal Olmayan Regresyon Modeli**

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [56]:
from warnings import filterwarnings
filterwarnings('ignore')

**KNN**

In [57]:
df = pd.read_csv("./dataset01.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division','NewLeague']])
y = df["Salary"]
X_ = df.drop (['Salary', 'League', 'Division', 'NewLeague'], axis = 1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25,random_state = 42)

In [58]:
X_train.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
183,328.0,91.0,12.0,51.0,43.0,33.0,2.0,342.0,94.0,12.0,51.0,44.0,33.0,145.0,59.0,8.0,1,0,1
229,514.0,144.0,0.0,67.0,54.0,79.0,9.0,4739.0,1169.0,13.0,583.0,374.0,528.0,229.0,453.0,15.0,1,0,1
286,593.0,152.0,23.0,69.0,75.0,53.0,6.0,2765.0,686.0,133.0,369.0,384.0,321.0,315.0,10.0,6.0,0,1,0
102,233.0,49.0,2.0,41.0,23.0,18.0,8.0,1350.0,336.0,7.0,166.0,122.0,106.0,102.0,132.0,10.0,0,0,0
153,341.0,95.0,6.0,48.0,42.0,20.0,10.0,2964.0,808.0,81.0,379.0,428.0,221.0,158.0,4.0,5.0,1,1,1


**Model işlemler**


In [59]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [61]:
knn_model

KNeighborsRegressor()

In [63]:
knn_model.n_neighbors

5

In [64]:
knn_model.metric

'minkowski'

In [65]:
dir(knn_model)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_check_feature_names',
 '_check_n_features',
 '_estimator_type',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_tree',
 '_validate_data',
 '_validate_params',
 '_y',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'feature_names_in_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params

In [66]:
knn_model.predict(X_test)[0:10]

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ,  325.5   ,
        216.5   ,  101.5   ,  982.    ,  886.6666])

**Tahmin işlemleri: Hata Karelerinin Ortalaması**

In [79]:
y_atama = knn_model.predict(X_test)

In [80]:
np.sqrt(mean_squared_error(y_test, y_atama))

421.6252180741266

**Model Tuning**

In [81]:
RMSE = []

for k in range(10):
    k += 1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(X_train, y_train)
    y_atama = knn_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_atama))
    RMSE.append(rmse)
    print("K = ",k,"İçin RMSE Değeri :", rmse)

K =  1 İçin RMSE Değeri : 455.03925390751965
K =  2 İçin RMSE Değeri : 415.99629571490965
K =  3 İçin RMSE Değeri : 420.6765370082348
K =  4 İçin RMSE Değeri : 428.8564674588792
K =  5 İçin RMSE Değeri : 426.6570764525201
K =  6 İçin RMSE Değeri : 423.5071669008732
K =  7 İçin RMSE Değeri : 414.9361222421057
K =  8 İçin RMSE Değeri : 413.7094731463598
K =  9 İçin RMSE Değeri : 417.84419990871265
K =  10 İçin RMSE Değeri : 421.6252180741266


**GridSearchCV Yöntemi**

In [82]:
knn_params = {"n_neighbors": np.arange(1,30,1)}

In [83]:
knn = KNeighborsRegressor()

In [84]:
knn_cv_model = GridSearchCV(knn, knn_params, cv = 10).fit(X_train, y_train)

In [85]:
knn_cv_model.best_params_

{'n_neighbors': 8}

**Final Model Test Hata Sonucu**

In [94]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"]).fit(X_train, y_train)

In [95]:
y_atama = knn_tuned.predict(X_test)

In [96]:
np.sqrt(mean_squared_error(y_test, y_atama))

413.7094731463598